In [1]:
import requests
import json
import time

/Users/ccmueller/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
FIND_URL = 'https://api.resy.com/4/find?lat=0&long=0&day={}&party_size=2&venue_id={}'
VENUE_URL = 'https://api.resy.com/3/venue/calendar?venue_id={}&num_seats=2&start_date=2020-01-01&end_date=2030-01-01'
SEARCH_URL = 'https://api.resy.com/3/venuesearch/search'

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'Authorization': 'ResyAPI api_key="VbWk7s3L4KiK5fzlO7JD3Q5EYolJI7n5"',
    'Accept': 'application/json, text/plain, */*',
    'Origin': 'https://resy.com',
    'X-Origin': 'https://resy.com'
}

### **Generate Restaurant Info**

In [ ]:
POINTS = [
    (40.712941, -74.006393), # Downtown
    (40.731248, -73.995177), # West Village
    (40.727896, -73.983021), # East Village
    (40.758192, -73.982890), # Midtown
    (40.785470, -73.976007), # Upper West
    (40.778061, -73.954034), # Upper East
    (40.810670, -73.946289), # Harlem
    (40.845711, -73.937796), # Washington Heights
    (40.816246, -73.918893), # South Bronx
    (40.769370, -73.911535), # Astoria
    (40.749186, -73.893048), # Jackson Heights
    (40.759380, -73.832923), # Flushing
    (40.710153, -73.960520), # Williamsburg
    (40.703422, -73.987805), # Dumbo
    (40.684090, -73.977800) # Downtown Bk
]

In [ ]:
slug_to_info = {}

for lat, lng in POINTS:
    response = requests.post(SEARCH_URL, headers=HEADERS, json={
        'page': 1,
        'per_page': 1000,
        'geo': {'latitude': lat, 'longitude': lng, 'radius': 35420},
        'query': ''
    })
    
    for hit in json.loads(response.text)['search']['hits']:
        slug_to_info[hit['url_slug']] = {
            'id': hit['id']['resy'],
            'name': hit['url_slug'],
            'cuisine': hit['cuisine'][0],
            'stars': str(hit['rating']['average'])[:4],
            'count': str(hit['rating']['count']),
            'where': hit['neighborhood'],
        }

In [ ]:
def count(response, word):
    return len([x for x in json.loads(response.text)['scheduled'] if x['inventory'] == word])

for slug, info in slug_to_info.items():
    try:
        response = requests.get(VENUE_URL.format(slug_to_info[slug]['id']), headers=HEADERS)
        info['available'] = str(count(response, 'available'))
        info['sold-out'] = str(count(response, 'sold-out'))
    except:
        print('Fetch failed for ' + slug)

In [ ]:
with open('resy.json', 'w') as fp:
    json.dump(slug_to_info, fp)

### **Scrape**

In [3]:
EMAIL = 'mariamabdullahedu@gmail.com'
SLUGS = [
    'sofreh'
]

In [4]:
with open('resy.json', 'r') as fp:
    slug_to_info = json.loads(fp.read())

In [12]:
import base64
import os.path

from email.message import EmailMessage
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def send_gmail(subject, content):
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('gmail', 'v1', credentials=creds)

        message = EmailMessage()
        message.set_content(content)
        message['To'] = EMAIL
        message['From'] = 'articasync@gmail.com'
        message['Subject'] = subject

        encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        create_message = {'raw': encoded_message}

        service.users().messages().send(userId='me', body=create_message).execute()
        
    except HttpError as error:
        print(error)

In [13]:
# Do not send something twice.
sent = {slug: set() for slug in SLUGS}

In [16]:
while True:
    for slug in SLUGS:
        resy_id = slug_to_info[slug]['id']
        venue_response = requests.get(VENUE_URL.format(resy_id), headers=HEADERS)
        for date in json.loads(venue_response.text)['scheduled']:
            if date['inventory'] != 'available':
                continue

            # CUSTOM FOR NATASHA
            if date['date'] != '2024-05-08':
                continue
        
            find_response = requests.get(FIND_URL.format(date['date'], resy_id), headers=HEADERS)
            for slot in json.loads(find_response.text)['results']['venues'][0]['slots']:
                day, hour = slot['date']['start'].split()
                if hour > '17' and hour < '21' and slot['date']['start'] not in sent[slug]:
                    try:
                        subject = 'Resybot: {} available on {} at {}'.format(slug, day, hour)
                        content = 'https://resy.com/cities/new-york-ny/venues/{}?date={}&seats=2'.format(slug, day)
                        send_gmail(subject, content)
                        sent[slug].add(slot['date']['start'])
                        print(subject)
                    except:
                        print('Failed to send gmail for {} on {} at {}'.format(slug, day, hour))

    # Sleep to not overwhelm the API.
    time.sleep(60)

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))